In [ ]:
from ultralytics import YOLO
import cv2
import math
import matplotlib.pyplot as plt
import pytesseract
import easyocr
import numpy as np

In [ ]:
# Cargar modelo
model = YOLO('yolov8n.yaml') # Construir nuevo modelo desde yaml

#results = model.train(data='spanish_license_plates_dataset/data.yaml', epochs=50, imgsz=640)


In [ ]:
def read_license_plate_easyocr(license_plate_crop):
    # Funcion para leer obtener como texto una matricula en una imagen

    # Carga del modelo de lengua Easyocr
    reader = easyocr.Reader(['en'], gpu=False) 

    # Reconocimiento de una imagen
    total_license = ""
    results = reader.readtext(license_plate_crop, min_size=70)
    for result in results:
        bbox, text, score = result
        text = text.upper().replace(' ', '')
        total_license += text
    return total_license


def read_license_plate_pytesseract(license_plate_crop):
    # Funcion para leer obtener como texto una matricula en una imagen

    # Carga del modelo de lengua Pytesseract
    pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract'

    # Lenguajes disponibles
    pytesseract.get_languages(config='')

    #Aplica reconocedor a imagen cargada
    return pytesseract.image_to_string(license_plate_crop)


Reconocedor de matrículas empleando imágenes

In [ ]:
# Carga el modelo con los pesos de la mejor epoca
bestModel = YOLO('runs/detect/train6/weights/best.pt')
classNames = ['matricula'] 

image_path = 'media/imagetest6.jpg'
img = cv2.imread(image_path)


img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

results = bestModel(img, stream=True)

for r in results:
    boxes = r.boxes

    # Matriculas detectadas
    for box in boxes:
        x1, y1, x2, y2 = box.xyxy[0]
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

        # Dibuja el contenedor y clase
        cv2.rectangle(img, (x1-10, y1-10), (x2+10, y2+10), (0, 0, 255), 4)

        # Imagen recortada de la matricula
        license_plate_crop = img[y1 - 10:y2 + 10, x1 - 10:x2 + 10]

        license_text = read_license_plate_easyocr(license_plate_crop)
        cv2.putText(img, license_text, [x1 -10, y1 -10], cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3)


plt.axis("off")
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)) 
plt.show()

Reconocedor de matrículas empleando videos

In [ ]:
# Load the YOLOv8 model
model = YOLO('runs/detect/train6/weights/best.pt')

# Open the video file
video_path = "media/prueba.mp4"
cap = cv2.VideoCapture(video_path)

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, conf=0.3, iou=0.5, persist=True)
        annotated_frame = results[0].plot()
        
        # Get the boxes and track IDs
        if results[0].boxes.id is not None:
            boxes = results[0].boxes.xyxy.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()

            for box in boxes:
                x1, y1, x2, y2 = map(int, np.array(box))
                print(f"x1: {x1}, y1: {y1}, x2: {x2}, y2: {y2}")
                
                # Imagen recortada de la matricula
                license_plate_crop = frame[y1:y2, x1:x2]

                gray_license_plate = cv2.cvtColor(license_plate_crop, cv2.COLOR_BGR2GRAY)
                _, binary_mask = cv2.threshold(gray_license_plate, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
                binary_mask = cv2.bitwise_not(binary_mask)
                
                license_text = read_license_plate_easyocr(binary_mask)
                cv2.putText(annotated_frame, license_text, [x1 -10, y1 -30], cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

                plt.imshow(cv2.cvtColor(binary_mask, cv2.COLOR_BGR2RGB))
                plt.axis('off')  # Turn off axis labels
                plt.show()

        # Display the annotated frame
        cv2.imshow("YOLOv8 Tracking", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()

In [ ]:
# Carga el modelo con los pesos de la mejor epoca
bestModel = YOLO('runs/detect/train6/weights/best.pt')
classNames = ['matricula']

# Carga el video
cap = cv2.VideoCapture('media/prueba.mp4')  
while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    results = bestModel(frame, stream=True)

    for r in results:
        boxes = r.boxes

        # Matriculas detectadas
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])

            cv2.rectangle(frame, (x1 - 10, y1 - 10), (x2 + 10, y2 + 10), (0, 0, 255), 4)
            # Imagen recortada de la matricula
            license_plate_crop = frame[y1 - 10:y2 + 10, x1 - 10:x2 + 10]
            license_text = read_license_plate_easyocr(license_plate_crop)
            cv2.putText(frame, license_text, [x1 -10, y1 -10], cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow('Video', frame)

    # Pulsar 'q' para salir del video
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam
cap.release()
cv2.destroyAllWindows()